# The task counts and calculations of the next month payments

In [20]:
# Importing libraries and packages
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Initialize WebDriver
driver = webdriver.Chrome()  # Add path to ChromeDriver if necessary
try:
    print("Opening browser...")
    driver.get("https://geoworkerz.telusinternational.ai/mapping/Timesheet/Pages/TaskCounts.aspx")


    # Wait for the page to load
    print("Waiting for tables to load...")
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.TAG_NAME, "table")))

    # Extract HTML
    print("Extracting page source...")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Find tables
    tables = soup.find_all('table')
    if tables:
        print(f"Found {len(tables)} tables:")
        for i, table in enumerate(tables):
            print(f"Table {i}: {table.get('id')}")
    else:
        print("No tables found.")

    dataframes = []
    for i, table in enumerate(tables[1:], start=2):  # Start from the second table
            rows = table.find_all('tr')
            table_data = []
            for row in rows:
                # Extract all cells (both <th> and <td>)
                cells = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
                table_data.append(cells)

            # Convert to DataFrame
            df = pd.DataFrame(table_data)
            print(f"Table {i}: Extracted {len(df)} rows.")
            dataframes.append(df)


            # Save to a CSV file
            df.to_csv("January.csv", index=False, header=False)
    
finally:
    driver.quit()

Opening browser...
Waiting for tables to load...
Extracting page source...
Found 2 tables:
Table 0: ctl00_ctl43_g_b5552829_fb25_4f4f_9fc6_b4e1ac0c59fd_gvTasks
Table 1: ctl00_ctl43_g_c2b46f33_ab26_4033_9f24_03936ae19b84_gvTasks
Table 2: Extracted 11 rows.


In [24]:
jt = []
jt = pd.read_csv('January.csv')
print(jt)

                              Task Type Allocated Time  Total Surveys Rated
0                      Audio Evaluation           180s                    4
1                 Audio File Comparison            60s                   56
2                 Audio File Comparison           180s                   66
3                     Fact Verification           240s                    7
4         Image to Caption Verification            60s                    1
5         Image to Caption Verification           300s                   21
6         Image to Caption Verification           650s                   15
7  Paired Images Attribute Verification           150s                    2
8             Photo Search Satisfaction           150s                   22
9                              sba_eval           200s                  127


In [26]:
# Changing the data type from string to integer
jt['Allocated Time'] = jt['Allocated Time'].str.replace('s', '').astype(int)

print(jt)

                              Task Type  Allocated Time  Total Surveys Rated
0                      Audio Evaluation             180                    4
1                 Audio File Comparison              60                   56
2                 Audio File Comparison             180                   66
3                     Fact Verification             240                    7
4         Image to Caption Verification              60                    1
5         Image to Caption Verification             300                   21
6         Image to Caption Verification             650                   15
7  Paired Images Attribute Verification             150                    2
8             Photo Search Satisfaction             150                   22
9                              sba_eval             200                  127


In [28]:
# Calculating the payment

jt['Total seconds'] = jt['Allocated Time'] * jt['Total Surveys Rated']
jt['Task Payment (US$)'] = jt['Total seconds']/60*0.26
jt['Task Payment (US$)'] = jt['Task Payment (US$)'].round(2)
jt_selected = jt.iloc[:, 1:]
print(jt_selected)

   Allocated Time  Total Surveys Rated  Total seconds  Task Payment (US$)
0             180                    4            720                3.12
1              60                   56           3360               14.56
2             180                   66          11880               51.48
3             240                    7           1680                7.28
4              60                    1             60                0.26
5             300                   21           6300               27.30
6             650                   15           9750               42.25
7             150                    2            300                1.30
8             150                   22           3300               14.30
9             200                  127          25400              110.07


In [30]:
# Adding the Total row
jt.iloc[-1,0 ] = 'Total'

In [32]:
# Calculating the summation of the columns
jt.iloc[1:, 2:5] = jt.iloc[1:, 2:5].astype(float)
jt.iloc[-1,1] = np.nan
for col in jt.columns[2:]:  # Skip the first two column 
    jt.loc[jt.index[-1], col] = jt[col].sum()
jt

,Task Type,Allocated Time,Total Surveys Rated,Total seconds,Task Payment (US$)
0,Audio Evaluation,180.0,4,720,3.12
1,Audio File Comparison,60.0,56,3360,14.56
2,Audio File Comparison,180.0,66,11880,51.48
3,Fact Verification,240.0,7,1680,7.28
4,Image to Caption Verification,60.0,1,60,0.26
5,Image to Caption Verification,300.0,21,6300,27.30
6,Image to Caption Verification,650.0,15,9750,42.25
7,Paired Images Attribute Verification,150.0,2,300,1.30
8,Photo Search Satisfaction,150.0,22,3300,14.30
9,Total,NaN,321,62750,271.92


In [34]:
print(f'the total payment of this month is : {jt.iloc[-1,-1]} $ = {jt.iloc[-1,-1]*1.4} CAD')

the total payment of this month is : 271.92 $ = 380.688 CAD
